In [1]:
import numpy as np
import sys
print(sys.path)
sys.path.append("/home/rgadea/experimentos/repositories/IC/")
sys.path.append("/home/rgadea/experimentos/repositories/ANTEA/")
import tables as tb
from invisible_cities.io import mcinfo_io
from invisible_cities.reco.xy_algorithms import barycenter
from antea.io import mc_io
import re
import os
from glob import glob
import h5py
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2


['', '/home/rgadea/anaconda3/envs/IC3.6/lib/python36.zip', '/home/rgadea/anaconda3/envs/IC3.6/lib/python3.6', '/home/rgadea/anaconda3/envs/IC3.6/lib/python3.6/lib-dynload', '/home/rgadea/anaconda3/envs/IC3.6/lib/python3.6/site-packages', '/home/rgadea/anaconda3/envs/IC3.6/lib/python3.6/site-packages/Sphinx-1.6.3-py3.6.egg', '/volumedisk0/home/rgadea/experimentos/repositories/IC', '/home/rgadea/anaconda3/envs/IC3.6/lib/python3.6/site-packages/IPython/extensions', '/volumedisk0/home/rgadea/.ipython']


ImportError: cannot import name 'units_dict'

In [2]:
dirname='/data/PETALO/'

In [3]:
def plot_2d(xarr, yval=None):
    x=range(xarr.shape[0])
    y=range(xarr.shape[1])
    fig = plt.figure(figsize=[15,10])
    
    if yval!=None:
        print('label: ',yval)
    #xy projection
    xx,yy=np.meshgrid(x,y)
    x1=xx.flatten()
    y1=yy.flatten()
    E1=xarr.flatten()
    indx=np.where(E1>0)
    E1=E1[indx]
    x1=x1[indx]
    y1=y1[indx]
    s=15*15
    ax1 = fig.add_subplot(111)#, projection='3d')
    sc3d = ax1.scatter(x1,y1,marker='s',cmap='jet',c=E1, s=s)
    ax1.set_xlim(0, 30)
    ax1.set_ylim(0, 30)
    fig.colorbar(sc3d)


### For now I decided to discard all events that got both gammas detected so I dont need to worry of separating 2 clusters of charges detected. I also discard events where interaction was not in the active volume, and that have less than 10 sipms lighted. The minimum threshold is 3 photons###

In [4]:
def get_events_and_cls_labels(mcinfo, evtnum):
    """Given mcinfo dictionary and event number cuts out events with 2 clusters and labels if gamma had compton scattering.
    Returns passed (0 or 1) and labels (0-no compton, 1-compton)"""
    pas=-1
    energy=0
    compt=0
    #make sure one of two mother gamma has 0 hits while the other has more than 0 hits
    for x in list(mcinfo[evtnum].keys()):
        if (mcinfo[evtnum][x].mother_indx==0) and (len(mcinfo[evtnum][x].hits)>0):
            pas*=-1
        #if (mcinfo[evtnum][x].mother_indx!=0) and (len(mcinfo[evtnum][x].hits)>0):
        #    energy+=mcinfo[evtnum][x].E
        if (mcinfo[evtnum][x].process=='compt') and (len(mcinfo[evtnum][x].hits)>0):
            compt=1
    return pas, compt

def get_ev_nums_and_labels(mcinfo):
    """Labels events for all event in mcinfo dictionary. Returns event numbers and corresponding labels."""
    evt_nums=[]; labels=[];
    for x in mcinfo.keys():
        pas,lab=get_events_and_cls_labels(mcinfo,x)
        if pas==1:
            evt_nums.append(x)
            labels.append(lab)
    return np.array(evt_nums), np.array(labels)    

#make dictionary of sensor positions
def get_pos_dict(table):
    """Returns dictionary of sensor_id: [x,y,z] """
    positions_table = table.root.MC.sensor_positions.read()
    keys=positions_table['sensor_id']
    vals=np.stack([positions_table['x'],positions_table['y'],positions_table['z']],axis=1)
    pos_dict=dict(zip(keys,vals))
    return pos_dict

def get_data_input_event(events, evnt,pos_dict, dimension_x=30,dimension_y=30, min_len=10, min_charge=2):
    """Given waveforms, event number  and information of sensor positions makes 2d image of sensor responses. 
    Cuts applied are:
    1. Minimum charge of sensor is larger than min_charge=2 photons
    2. Only events that have more min_len=10 are accounted
    3. The dimension of the image is dimension_x=30,dimension_y=30 around the barycenter of the event.
    Returns 0,1 value if event passed the cut and the voxelized image."""
    pas=0
    tensor=np.zeros((dimension_x,dimension_y))
    try:
        data=np.array([np.append(pos_dict[x],events[evnt][x].charges) for x in events[evnt].keys() if  (events[evnt][x].charges[0]>min_charge)] )        
    except IndexError:
        return pas,tensor
    
    if len(data)>min_len:
        data=np.array(list(map(lambda x: [np.arctan2(x[1],x[0]),x[2],x[3]],data)))
        if (data[:,0].max()-data[:,0].min())>3:
            data[:,0]=np.mod((data[:,0]+np.pi/2),2*np.pi)
        barcnt=barycenter(data[:,0:2], data[:,2])
        Xc,Yc=barcnt[0].X,barcnt[0].Y
        data[:,0]=np.round((data[:,0]-Xc)/4*129.4)
        data[:,1]=np.round((data[:,1]-Yc)/4)
        tensor=np.zeros((dimension_x,dimension_y))
        for d in data:
            indx_i=int(d[0]+dimension_x/2)
            indx_j=int(d[1]+dimension_y/2)
            try:
                tensor[indx_i,indx_j]=d[2]
            except IndexError:
                pass
        pas=1
    return pas,tensor

In [5]:
def get_input_and_labels(input_file, return_mcinfo=False):
    """ Given input file it makes 2d normalized images of sensor responses and labels for the events that pass the cuts.
    If return_mcinfo=True returns true MC information (for checks)."""
    table=tb.open_file(input_file, mode='r')
    mcinfo=mcinfo_io.read_mcinfo(table)
    print('read mc info')
    evt_nums, labels=get_ev_nums_and_labels(mcinfo)
    pos_dict=get_pos_dict(table)
    #read waveforms
    allevs=mc_io.read_mcsns_response(input_file)
    print('read waveforms')
    x_arr=[];ev_arr=[]; y_arr=[]
    for ev in allevs.keys():
        if ev in evt_nums:
            pas, data=get_data_input_event(allevs, ev,pos_dict)
            if pas==1:
                data=data/data.sum()
                x_arr.append(data)
                ev_arr.append(ev)
                y_arr.append(labels[evt_nums==ev])
    x_arr=np.array(x_arr)
    y_arr=np.array(y_arr)
    ev_arr=np.array(ev_arr)
    x_arr=np.reshape(x_arr, (x_arr.shape[0], x_arr.shape[1], x_arr.shape[2],1))
    y_arr=np.reshape(y_arr, (y_arr.shape[0],1))
    if return_mcinfo:
        return x_arr, y_arr,ev_arr, mcinfo
    else:
        return x_arr, y_arr,ev_arr
    

In [6]:
def get_input_and_labels_files(directory, num_files):
    """ Makes training data for NN for num_files from directory."""
    files=sorted(os.listdir(dirname),key=lambda f:[int(x) for x in re.findall(r'\d+', f)])[:num_files]
    x_arr=[];y_arr=[];ev_arr=[]
    for f in files:
        input_file=dirname+f
        x,y,e=get_input_and_labels(input_file)
        x_arr.append(x)
        y_arr.append(y)
        ev_arr.append(e)
    x_arr=np.concatenate(x_arr)
    y_arr=np.concatenate(y_arr)
    ev_arr=np.concatenate(ev_arr)
    return x_arr,y_arr,ev_arr
        

## example how to read and plot events from one file ##

In [7]:
files=sorted(os.listdir(dirname),key=lambda f:[int(x) for x in re.findall(r'\d+', f)])
input_file=dirname+files[0]
x,y,e,mcinfo=get_input_and_labels(input_file, return_mcinfo=True)

NameError: name 'os' is not defined

In [8]:
%matplotlib inline
indx=np.random.choice(range(x.shape[0]))
print('event number: ',e[indx])
plot_2d(x[indx],y[indx])

NameError: name 'x' is not defined

In [ ]:
mcinfo[e[indx]]

### example how to create training set form 2 files ###

In [ ]:
x_train,y_train,ev_train=get_input_and_labels_files(dirname, 2)